In [1]:
import sys

sys.path[1]

'/home/knguyen02311/Datadisk_link/Programming/Python/AIBots'

In [2]:
from os import environ
from dotenv import load_dotenv

load_dotenv(dotenv_path='/home/knguyen02311/Datadisk_link/Programming/Python/AIBots/backend/.env.dev', verbose=True)
print(environ.get("POSTGRES_HOST"))


class EnvPostgres:
    HOST = environ.get("POSTGRES_HOST")
    DB = environ.get("POSTGRES_DB")
    USER = environ.get("POSTGRES_USER")
    PASSWORD = environ.get("POSTGRES_PASSWORD")
    PORT = int(environ.get("POSTGRES_PORT"))
    URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB}?sslmode=disable"


class EnvRedis:
    HOST = environ.get("REDIS_HOST")
    PORT = int(environ.get("REDIS_PORT"))
    PASSWORD = environ.get("REDIS_PASSWORD")
    DB = int(environ.get("REDIS_DB"))


class EnvSQLAlchemy:
    ECHO = True
    AUTO_FLUSH = True  # flush after committing
    AUTO_COMMIT = False
    POOL_SIZE = 15
    MAX_OVERFLOW = 10
    POOL_PRE_PING = False


localhost


In [3]:
import redis
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# For PostgresSQL
POSTGRES_ENGINE = create_engine(url=EnvPostgres.URL,
                                echo=EnvSQLAlchemy.ECHO,
                                pool_size=EnvSQLAlchemy.POOL_SIZE,
                                max_overflow=EnvSQLAlchemy.MAX_OVERFLOW,
                                pool_pre_ping=EnvSQLAlchemy.POOL_PRE_PING)

POSTGRES_SESSION_FACTORY = sessionmaker(bind=POSTGRES_ENGINE,
                                        autoflush=EnvSQLAlchemy.AUTO_FLUSH,
                                        autocommit=EnvSQLAlchemy.AUTO_COMMIT)

In [4]:
from datetime import datetime
from typing import Annotated, Optional, List
from sqlalchemy import ForeignKey, types, Table, Column, Integer
from sqlalchemy.orm import Mapped, mapped_column, relationship, DeclarativeBase, MappedAsDataclass

str64 = Annotated[str, None]
str256 = Annotated[str, None]

int_PK = Annotated[int, mapped_column(primary_key=True)]
smallint = Annotated[int, None]
timestamp = Annotated[datetime, mapped_column(default=datetime.utcnow)]


class Base(DeclarativeBase):
    type_annotation_map = {
        str64: types.VARCHAR(64),
        str256: types.VARCHAR(256),
        smallint: types.SMALLINT,
        timestamp: types.TIMESTAMP,
    }


class Test(Base):
    __tablename__ = "test"
    id: Mapped[int_PK]
    name: Mapped[str64]


In [5]:
Base.metadata.create_all(POSTGRES_ENGINE)

2024-01-29 13:06:38,215 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-01-29 13:06:38,216 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-29 13:06:38,218 INFO sqlalchemy.engine.Engine select current_schema()
2024-01-29 13:06:38,219 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-29 13:06:38,220 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-01-29 13:06:38,221 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-29 13:06:38,223 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-29 13:06:38,229 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [6]:
with POSTGRES_SESSION_FACTORY() as session:
    test = Test(name="test")
    session.add(test)
    session.commit()
    print(test.id)

2024-01-29 13:06:40,481 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-29 13:06:40,484 INFO sqlalchemy.engine.Engine INSERT INTO test (name) VALUES (%(name)s) RETURNING test.id
2024-01-29 13:06:40,485 INFO sqlalchemy.engine.Engine [generated in 0.00118s] {'name': 'test'}
2024-01-29 13:06:40,487 INFO sqlalchemy.engine.Engine COMMIT
2024-01-29 13:06:40,559 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-29 13:06:40,562 INFO sqlalchemy.engine.Engine SELECT test.id AS test_id, test.name AS test_name 
FROM test 
WHERE test.id = %(pk_1)s
2024-01-29 13:06:40,562 INFO sqlalchemy.engine.Engine [generated in 0.00070s] {'pk_1': 2}
2
2024-01-29 13:06:40,564 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
from sqlalchemy import select, update, delete

with POSTGRES_SESSION_FACTORY() as session:
    updatequery = update(Test).returning(Test).where(Test.id == 1).values(name="test2")
    result = session.scalar(updatequery)
    session.commit()
    print(result)

2024-01-29 13:07:05,088 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-29 13:07:05,089 INFO sqlalchemy.engine.Engine UPDATE test SET name=%(name)s WHERE test.id = %(id_1)s RETURNING test.id, test.name
2024-01-29 13:07:05,090 INFO sqlalchemy.engine.Engine [cached since 20.84s ago] {'name': 'test2', 'id_1': 1}
2024-01-29 13:07:05,093 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
with POSTGRES_SESSION_FACTORY() as session:
    session.

NoInspectionAvailable: No inspection system is available for object of type <class 'str'>